In [1]:
import requests, math, re, sys, time, json
from bs4 import  BeautifulSoup as bs

In [2]:
rs = requests.session()
res = rs.get('http://vacation.eztravel.com.tw/pkgfrn/results/TPE/49?fullStatus=NONE')
soup = bs(res.text, "lxml")

## 抓取頁數 & 抓取連結

In [3]:
count  = soup.select('.v-line')[0].text.split(' ') #抓出有幾筆資料
print count[1]
page = int(math.ceil(float(count[1]) / 18)) #算出總頁數
print page

119
7


In [4]:
urlFormat = 'http://vacation.eztravel.com.tw/pkgfrn/results/TPE/49/{}?&fullStatus=NONE' #建立換頁的網址字串
with open('D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt','w') as f:
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt
    for pageNum in range(1, page + 1): #從第一頁到第七頁的前一頁停止，故再多加一
        url = urlFormat.format(pageNum)
        res2 = rs.get(url)
        soup1 = bs(res2.text, "lxml")
        list_table = soup1.select('div#listTable')[0]
        for items in list_table.select('div.list-box.mainLinkBox'):
            f.write('http://vacation.eztravel.com.tw/pkgfrn/' + items.select('a')[0]['href'].split('/pkgfrn/')[1] + '\n')
            #依照內容規則變化，加上字串相加，將網址寫入到txt檔中

## 檢查是否有重複的key與value

In [4]:
str = 'http://vacation.eztravel.com.tw/pkgfrn/introduction/FRN0000014050/20160524'
get = str.split('/')
print get

['http:', '', 'vacation.eztravel.com.tw', 'pkgfrn', 'introduction', 'FRN0000014050', '20160524']


In [5]:
dicCheck = {} #宣告一個dic，稍後會將檔名(Key)跟網址(Value)一一放入
num = 0
for bid_url in open('D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt', 'r'): #讀連結檔
    num += 1
    splitStr = bid_url.split('/') #將網址切割，並且丟入splitStr這個List裡
    linkName = splitStr[5].strip() + "_" + splitStr[6].strip() #將List[index]對應到的元素，以字串型式組合成dataNmae
    #用商品編號當作檔名
    if linkName not in dicCheck:
        print 'No.' + str(num) + ', ' + linkName +  ' is new, and added in dicCheck.'
        dicCheck[linkName] = bid_url
    else:
        print 'No.' + str(num) + ', ' + linkName + ' is existed, and not added in dicCheck.'
        print linkName, dicCheck[linkName] #若有重複，則print出重複的Key跟Value

No.1, VDR0000001943_NGO04IT6613H is new, and added in dicCheck.
No.2, FRN0000006441_20160603 is new, and added in dicCheck.
No.3, VDR0000001888_NGO04160614A is new, and added in dicCheck.
No.4, VDR0000001943_NGO04IT6904H is new, and added in dicCheck.
No.5, VDR0000001279_DTS-A0613NN2 is new, and added in dicCheck.
No.6, VDR0000007986_NGO05CI61416C_485 is new, and added in dicCheck.
No.7, VDR0000001846_16JP605CIA is new, and added in dicCheck.
No.8, VDR0000001913_TOM05IT160617I is new, and added in dicCheck.
No.9, VDR0000001943_NGO04IT6703H is new, and added in dicCheck.
No.10, VDR0000001975_KMQ04160927TA is new, and added in dicCheck.
No.11, VDR0000001943_NRT05IT6612D is new, and added in dicCheck.
No.12, FRN0000014083_20160622 is new, and added in dicCheck.
No.13, VDR0000001888_NGO05160622A is new, and added in dicCheck.
No.14, VDR0000001914_NGO05160903L is new, and added in dicCheck.
No.15, VDR0000001975_KMQ05160623TA is new, and added in dicCheck.
No.16, VDR0000001975_KMQ04161116TA 

## 將需要的資料抓入檔案

In [5]:
### 印出字典的key與value########
def PrintKeyValue(dic_in):
    for key, value in dic_in.iteritems():
        print key, ':', value

In [12]:
eachTravel = [] #放置所有細項資料於旅行方案集合
totalTravel = {} #放置所有的旅行方案於總集合
totalCount = 0
errorCount = 0
with open('D:\\Big Data\\Project\\Travel\\EZTravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Shikoku.txt', 'w') as errorFile:
    #寫error檔
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Shikoku.txt
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Shikoku.txt
    for bid_url in open('D:\\Big Data\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt', 'r'):
        #讀連結檔
        #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt
        #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt
        try: #使用try-except
            itemDetail = {} #放置所有細項資料集合
            daysTour = {} #放置day{}的value
            bid_detail = requests.get(bid_url.strip()) #讀取檔案中的連結
            soup2 = bs(bid_detail.text, "lxml") #萃取資料
            row = soup2.select('div#pkgfrnVisitHistory')[0]
            agency = soup2.select('div.container.tbShow')[0].text.strip().split()
            price = re.search('\d+', row.select('.price')[0].text) #利用正規表示法抓取價格
            plane = re.search('.\W{2,4}', row.select('div#flight-block1 span')[3].text) #利用正規表示法抓取航空公司名稱
            dayBlock = soup2.select('ol.day-block')[0] #抓取天數
            
            itemDetail['title'] = row.select('div.pro-title.css-td h1')[0].text #將個別資料丟入itemDetail字典中，以下相同
            itemDetail['agency'] = agency[0]
            itemDetail['prodNo'] = row.select('div.pro-id-right.css-td span')[0].text.strip() + ' ' + row.select('div.pro-id-right.css-td span')[1].text.strip()
            itemDetail['href'] = bid_url.strip()
            #因為EZtravel的網站資料較鬆散，且只有旅館跟行程有規律性，故特別獨立出來寫加入的方法
            daysTourFormat = 'day{}' #建立字串格式
            for indexTour in range(0,len(dayBlock.select('div.title-circle.title-circle-green.nth-day span'))):
                daysTourInput = daysTourFormat.format(dayBlock.select('div.title-circle.title-circle-green.nth-day span')[indexTour].text) #這邊是天數，day{1}_Tour、day{2}_Tour……
                daysTour[daysTourInput] = row.select('div.day-box h4')[indexTour].text.strip()
                #這邊是索引值，row.select('div.day-box h4')[1].text.strip()、row.select('div.day-box h4')[2].text.strip()……
            itemDetail['tour'] = daysTour
            totalCount += 1 #計算有多少筆資料丟入JSON檔中

            print '----------------------------這是分隔線----------------------------'
            print 'days: ' + dayBlock.select('div.title-circle.title-circle-green.nth-day span')[-1].text #印出當前旅行方案的旅行天數
            print 'Url: ' + bid_url.strip()
#             PrintKeyValue(daysTour)
#             PrintKeyValue(itemDetail)
            eachTravel.append(itemDetail)
            totalTravel['eachTravel'] = eachTravel

        except:
            errorCount += 1
            splitPID = bid_url.strip().split('/')
            strPID = splitPID[5].strip() + "_" + splitPID[6].strip()
            #當錯誤訊息出現時，印出是哪一筆資料出現錯誤
            print '----------------' + strPID + ' is error!!----------------'
                #將出現錯誤的資料網址，寫入txt檔中
            errorFile.write(bid_url)

with open('D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Shikoku.json', 'w') as dataFile:
    #寫最後全部資料檔
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Shikoku.json
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Shikoku.json
    json.dump(totalTravel, dataFile)

print '----------------------------All datas are done!!!----------------------------'
print 'Data Quantities: ' + str(totalCount)
print 'Error Quantities: ' + str(errorCount)

----------------------------這是分隔線----------------------------
days: 4
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000001943/NGO04IT6613H
----------------------------這是分隔線----------------------------
days: 4
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/FRN0000006441/20160603
----------------------------這是分隔線----------------------------
days: 4
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000001888/NGO04160614A
----------------------------這是分隔線----------------------------
days: 4
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000001943/NGO04IT6904H
----------------------------這是分隔線----------------------------
days: 5
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000001279/DTS-A0613NN2
----------------VDR0000007986_NGO05CI61416C_485 is error!!----------------
----------------------------這是分隔線----------------------------
days: 5
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000001846/16JP605CIA
-------

## JSON LINK: http://stackoverflow.com/questions/7100125/storing-python-dictionaries

In [10]:
with open(D':\\Big 
          Data\\Project\\Travel\\EZTravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Shikoku.json', 'r') as fileRead:
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Shikoku.json
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Shikoku.json
    data = json.load(fileRead)
    print data

IOError: [Errno 2] No such file or directory: 'E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Shikoku.json'

## 用不到的Code暫放處

In [11]:
#             daysHotFormat = 'day{}_Hotel' #建立字串格式
#             itemDetail['Price'] = price.group(0)
#             itemDetail['City'] = row.select('.tag')[0].text.strip()
#             itemDetail['Time'] = row.select('div#flight-block1 span')[2].text.strip()
#             itemDetail['PlaneCom'] = plane.group(0)
#             if len(row.select('div.pro-id-right.css-td span')[0].text.strip()) > 0: #判斷若PID_1長度大於零，則將資料丟入
#                 itemDetail['PID_1'] = row.select('div.pro-id-right.css-td span')[0].text.strip()
#             else: #判斷若PID_1長度小於零，則丟入Null
#                 itemDetail['PID_1'] = u'null'
#             if len(row.select('div.pro-id-right.css-td span')[1].text.strip()) > 0: #同上
#                 itemDetail['PID_2'] = row.select('div.pro-id-right.css-td span')[1].text.strip()
#             else: #同上
#                 itemDetail['PID_2'] = u'null'
#             num = 1 #因住宿飯店的索引值是[1]、[3]、[5]……的奇數，所以給起始值，每執行一次會增加2
#             for times in range(0,int(days.group(0))):
#                 daysHotInput = daysHotFormat.format(times + 1) #這邊是天數，day{1}_Hotel、day{2}_Hotel……
#                 itemDetail[daysHotInput] = row.select('div.meals')[num].text.strip()
#                 #這邊是索引值(奇數)，row.select('div.meals')[1].text.strip()、row.select('div.meals')[3].text.strip()……
#                 num += 2
#             print 'Url: ' + bid_url.strip() #印出當前旅行方案的網址